<a href="https://colab.research.google.com/github/CharlottePrimiceri/VP_Project/blob/main/MaskR_CNN_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
!pip install torchvision --quiet

In [ ]:
! pip install scikit-image --quiet

In [ ]:
! pip install opencv-python --quiet

In [ ]:
! pip install Pillow

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Cityscapes Conversion to COCO Format

In [ ]:
! git clone https://github.com/ISSResearch/Dataset-Converters

In [ ]:
! pip install -r /content/Dataset-Converters/requirements.txt

In [ ]:
! python /content/Dataset-Converters/convert.py -i /content/drive/MyDrive/VPPROJECT/cityscapes -o /content/drive/MyDrive/VPPROJECT/cityscapes/annotations -I CITYSCAPES -O COCO --copy

# Filtering 3 Classes: Person, Car and Bicycle

In [ ]:
filtered_annotations_file = "filtered_annotations.json"

# Specify the IDs of the desired classes to keep
classes = [1, 2, 3]

# Load the COCO annotations
with open("/content/drive/MyDrive/VPPROJECT/cityscapes/annotations/train.json", "r") as f:
    data = json.load(f)

# Filter the annotations for the desired classes
filtered_annotations = {
    "images": data["images"],
    "categories": [category for category in data["categories"] if category["id"] in desired_class_ids],
    "annotations": [annotation for annotation in data["annotations"] if annotation["category_id"] in desired_class_ids]
}

# Save the filtered annotations to a new JSON file
with open(filtered_annotations_file, "w") as f:
    json.dump(filtered_annotations, f)

In [ ]:
filtered_annotations_file = "filtered_annotations_val.json"

# Load the COCO annotations
with open("/content/drive/MyDrive/VPPROJECT/cityscapes/annotations/val.json", "r") as f:
    data = json.load(f)

filtered_annotations = {
    "images": data["images"],
    "categories": [category for category in data["categories"] if category["id"] in desired_class_ids],
    "annotations": [annotation for annotation in data["annotations"] if annotation["category_id"] in desired_class_ids]
}

with open(filtered_annotations_file, "w") as f:
    json.dump(filtered_annotations, f)

In [ ]:
with open('/content/drive/MyDrive/VPPROJECT/cityscapes/annotations/train.json') as json_data:
    data = json.load(json_data)
    df3_pandas = pd.DataFrame(data['categories'])
df3_pandas
list_of_values = [1, 3, 8]
df3 = df3_pandas[df3_pandas['id'].isin(list_of_values)]
print(df3)
df3_json = df3.to_dict(orient='dict')

# Detectron2 Configuration on Cityscapes

In [ ]:
! git clone https://github.com/facebookresearch/detectron2

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"

In [ ]:
DatasetCatalog.clear()

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/VPPROJECT/cityscapes/annotations/filtered_annotations.json", "/content/drive/MyDrive/VPPROJECT/cityscapes/annotations/train")

In [ ]:
nuts_metadata = MetadataCatalog.get('my_dataset_train')
dataset_dicts = DatasetCatalog.get('my_dataset_train')

# Visualize Original Image and Prediction

In [ ]:
im = cv2.imread("/content/drive/MyDrive/VPPROJECT/cityscapes/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png")
cv2_imshow(im)

In [ ]:
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])